### Analyze outliers in detection/prediction results on test data

In [7]:
import pandas as pd
import numpy as np
import os

In [22]:
def read_prediction_dataframe_from_dir(folder: str, is_tracking=False) -> pd.DataFrame:
    file_list = sorted(os.listdir(folder))
    dfs = []
    for i, file_name in enumerate(file_list):
        #print(f)
        columns=["cls", "x", "y", "w", "h", "conf", "id"] if is_tracking else ["cls", "x", "y", "w", "h", "conf"]

        df = pd.read_csv(os.path.join(folder, file_name), sep=" ", header=None, names=columns)
        df["frame"] = i
        df["filename"] = file_name
        keypoints_df = df[df["cls"] >= 31]
        #print(keypoints_df)
        dfs.append(keypoints_df)

    return pd.concat(dfs)

# 13-classes keypoints model

### Pony vs The Killjoy - Pool 013
#### 2 keypoints classes variant: cone (31), midpoint(32)
Loading prediction/tracking data

In [23]:
#print(os.getcwd())
keypoints_df = read_prediction_dataframe_from_dir(folder="../../runs/detect/track3/labels", is_tracking=True)
print(len(keypoints_df))
keypoints_df.head(5)

55810


,cls,x,y,w,h,conf,id,frame,filename
3,31,0.699340,0.103258,0.009634,0.017423,0.943481,4,0,pony_vs_the_killjoys_pool_013_1.txt
7,32,0.518308,0.306945,0.014788,0.026235,0.909432,8,0,pony_vs_the_killjoys_pool_013_1.txt
9,31,0.342385,0.105246,0.009379,0.017071,0.889964,10,0,pony_vs_the_killjoys_pool_013_1.txt
10,32,0.519433,0.232265,0.011592,0.020488,0.887180,11,0,pony_vs_the_killjoys_pool_013_1.txt
11,31,0.311012,0.136410,0.008984,0.016713,0.880367,12,0,pony_vs_the_killjoys_pool_013_1.txt


In [10]:
conf_threshold = 0.5
cones_threshold = 4

In [30]:
def get_ratio_of_frames_with_less_than_n_cones(df: pd.DataFrame, n: int=4, conf_threshold=conf_threshold) -> float:
    df_filtered = df.groupby('frame').filter(lambda group: len(group[((group["cls"]==31) & (group["conf"] > conf_threshold))]) < n)
    #print(df_filtered['frame'].head())
    return len(df_filtered['frame'].unique())/len(df['frame'].unique())

In [31]:
ratio = get_ratio_of_frames_with_less_than_n_cones(keypoints_df, n=cones_threshold, conf_threshold=0.6)
print(f"Percent of frames with less than {cones_threshold} keypoints with confidence threshold {conf_threshold*100} % --> {ratio*100} %")

Percent of frames with less than 4 keypoints with confidence threshold 50.0 % --> 18.90265076976502 %


In [26]:
ponys_keypoints_df = keypoints_df.copy()
if "filename" in ponys_keypoints_df.columns:
    del ponys_keypoints_df["filename"]
ponys_counts_by_frame = ponys_keypoints_df[ponys_keypoints_df["cls"]==31].groupby("frame")["id"].aggregate('count')
# ponys_keypoints_df = ponys_keypoints_df.rolling(window=10).max()
# ponys_keypoints_df[ponys_keypoints_df < 4]
#ponys_counts_by_frame
ponys_counts_by_frame_series = ponys_counts_by_frame.rolling(window=15).max()
ponys_counts_by_frame_series[ponys_counts_by_frame_series < 4]

Series([], Name: id, dtype: float64)

In [14]:
keypoints_df.head()

,cls,x,y,w,h,conf,id,frame,filename
3,31,0.699340,0.103258,0.009634,0.017423,0.943481,4,0,pony_vs_the_killjoys_pool_013_1.txt
7,32,0.518308,0.306945,0.014788,0.026235,0.909432,8,0,pony_vs_the_killjoys_pool_013_1.txt
9,31,0.342385,0.105246,0.009379,0.017071,0.889964,10,0,pony_vs_the_killjoys_pool_013_1.txt
10,32,0.519433,0.232265,0.011592,0.020488,0.887180,11,0,pony_vs_the_killjoys_pool_013_1.txt
11,31,0.311012,0.136410,0.008984,0.016713,0.880367,12,0,pony_vs_the_killjoys_pool_013_1.txt


### Machine vs Condors - Pool 006
#### 2 keypoints classes variant: cone (31), midpoint(32)
Loading prediction/tracking data

In [15]:
condors_keypoints_df = read_prediction_dataframe_from_dir(folder="../../runs/detect/track2/labels", is_tracking=True)
print(len(condors_keypoints_df))
condors_keypoints_df.head()

19090


,cls,x,y,w,h,conf,id,frame,filename
3,32,0.500280,0.276708,0.010053,0.017431,0.955173,4,0,machine_vs_condors_pool_006_1.txt
8,32,0.501809,0.323580,0.010426,0.018345,0.941980,9,0,machine_vs_condors_pool_006_1.txt
13,31,0.299682,0.233273,0.009490,0.017113,0.904370,14,0,machine_vs_condors_pool_006_1.txt
14,31,0.698093,0.229010,0.010256,0.017395,0.889027,15,0,machine_vs_condors_pool_006_1.txt
16,31,0.327827,0.200379,0.009752,0.016872,0.867725,17,0,machine_vs_condors_pool_006_1.txt


In [16]:
ratio = get_ratio_of_frames_with_less_than_n_cones(condors_keypoints_df, n=4, conf_threshold=0.5)
print(f"Percent of frames with less than {cones_threshold} keypoints with confidence threshold {conf_threshold*100} % --> {ratio*100} %")

Percent of frames with less than 4 keypoints with confidence threshold 50.0 % --> 32.686868686868685 %


In [17]:
condors_counts_by_frame = condors_keypoints_df[condors_keypoints_df["cls"]==31].groupby("frame")["id"].aggregate('count')
window_len = 10
cones_threshold = 4
condors_smoothed_series = condors_counts_by_frame.rolling(window=window_len).max()
print(f"Number of images with fewer than {4} cones inside rolling window of length={window_len}: {len(condors_smoothed_series[condors_smoothed_series < cones_threshold])}")

Number of images with fewer than 4 cones inside rolling window of length=10: 68


# 13-classes keypoints model

### Pony vs The Killjoy - Pool 013
#### 13 keypoints classes variant
Loading prediction/tracking data

In [27]:
ponys_13cls_df = read_prediction_dataframe_from_dir(folder="../../runs/detect/track4/labels", is_tracking=True)
print(len(ponys_13cls_df))
ponys_13cls_df.head(5)

22643


,cls,x,y,w,h,conf,id,frame,filename
2,36,0.518324,0.271519,0.009460,0.016470,0.969942,3,0,pony_vs_the_killjoys_pool_004_1.txt
4,34,0.321885,0.224127,0.008479,0.015690,0.965040,5,0,pony_vs_the_killjoys_pool_004_1.txt
11,33,0.522141,0.206899,0.008848,0.015378,0.927504,12,0,pony_vs_the_killjoys_pool_004_1.txt
12,31,0.352514,0.190828,0.008629,0.014922,0.900049,13,0,pony_vs_the_killjoys_pool_004_1.txt
14,32,0.694432,0.191096,0.008541,0.015429,0.889788,15,0,pony_vs_the_killjoys_pool_004_1.txt


Eligible pairs of keypoints that run on parallel lines:
- 31, 32 (TLC,TRC)
- 34, 35 (TLF, TRF)
- 39, 40 (BLF, BRF)
- 41, 42 (BLC, BRC; practically never visible)

In [54]:
candidate_clsid_pairs = np.array([[31,32],[34,35],[39,40],[41,42]])
keypoint_lines_map = {31: "TC", 32: "TC", 34: "TF", 35: "TF", 39: "BF", 40: "BF", 41: "BC", 42: "BC"}
single_frame_df = ponys_13cls_df[ponys_13cls_df["frame"]==0]
single_frame_df = single_frame_df[(single_frame_df["cls"].isin(candidate_clsid_pairs.flatten())) & (single_frame_df["conf"] > conf_threshold)]
single_frame_df["keypoint_line"] = single_frame_df["cls"].apply(lambda c: keypoint_lines_map[c])
keypoint_counts = single_frame_df.groupby("keypoint_line")["cls"].aggregate("count")
print(len(keypoint_counts[keypoint_counts > 1]))
keypoint_counts[keypoint_counts > 1]
#keypoint_counts[0]
#keypoint_counts[keypoint_counts["count"]]
# mask1 = (group["cls"].isin([31,32])) & (group["conf"] > conf_threshold)
# mask2 = (group["cls"].isin([34,35])) & (group["conf"] > conf_threshold)
# mask2 = (group["cls"].isin([39,40])) & (group["conf"] > conf_threshold)

#group[mask]

2


keypoint_line
TC    2
TF    2
Name: cls, dtype: int64

In [72]:
def get_number_of_keypoint_pairs(group: pd.DataFrame) -> int:
    mask = (group["cls"].isin(candidate_clsid_pairs.flatten())) & (group["conf"] > conf_threshold)
    candidate_cls_df = group[mask].copy(deep=False)
    #df2 = group.loc[mask, :]
    candidate_cls_df["keypoint_line"] = candidate_cls_df["cls"].apply(lambda c: keypoint_lines_map[c])
    keypoint_counts = candidate_cls_df.groupby("keypoint_line")["cls"].aggregate("count")
    return len(keypoint_counts[keypoint_counts >= 2])

def get_ratio_of_frames_with_fewer_than_2x2_keypoints(df: pd.DataFrame, n: int=2, conf_threshold=conf_threshold) -> float:
    df_filtered = df.groupby('frame').filter(lambda g: get_number_of_keypoint_pairs(g) < n)
    return len(df_filtered['frame'].unique())/len(df['frame'].unique())

In [73]:
ratio = get_ratio_of_frames_with_fewer_than_2x2_keypoints(ponys_13cls_df, n=2, conf_threshold=0.6)
print(f"Percent of frames with less than 2 pairs of keypoints with confidence threshold {conf_threshold*100} % --> {ratio*100} %")

Percent of frames with less than 2 pairs of keypoints with confidence threshold 50.0 % --> 4.192229038854806 %
